In [42]:
#Load libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import geopy
from geopy.distance import vincenty
from geopy.distance import VincentyDistance
import srtm
from geohelper import distance 
from geohelper import bearing
import pandas as pd
import psycopg2
import sys
import pandas
import numpy as np
import math
from pandas.io import sql
import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.types import INTEGER
from datetime import datetime
import urllib
from urllib import request
import json
import os
from configparser import ConfigParser
import contextlib

In [138]:
## define paths and variables

sql_path =  "/home/jovyan/rural_planner_refactoring/sql/ar"
config_path = "/home/jovyan/rural_planner_refactoring/config_files/config_ar"

schema          = 'rural_planner_dev'

output_table = 'transport_by_tower_all'
temporary_table = 'temp_transport_by_tower_all'
table_infrastructure = 'infrastructure_global'

mw_radius = 40000 
fiber_radius = 2000
los_interval = 20
sources_omit = ['CLARO','CLARO_POINTS','PERSONAL','PERSONAL_POINTS']
owners_tx = {
    "tasa" : {
        "owners" : ['TASA','TASA_FIXED'],
        "owner_name" : 'movistar'
    },
    "arsat" : {
        "owners" : ['ARSAT'],
        "owner_name" : 'arsat'
    },
    "silica" : {
        "owners" : ['SILICA'],
        "owner_name" : 'silica'
    },
    "gigared" : {
        "owners" : ['GIGARED'],
        "owner_name" : 'gigared'
    },
    "fiber_points" : {
        "owners" : ['EPEC_POINTS', 
                             'FIBRA_PROV_SAN_LUIS_POINTS', 'HG_PISADA_SION_POINTS', 
                             'SION_USHUAIA_POINTS', 'TELMEX_POINTS'],
        "owner_name" : 'points'
    }      
}

third_party_owners = ['arsat','silica','gigared','points']
regional_owners = []

parser = ConfigParser()
parser.read(config_path)

apikey = parser.get('db_config', 'apikey')

In [131]:
#Define functions: Line of sight, elevation_google and print_full
## Lambda set for freq=11GHz

def check_los(origin, destination, tower_height_1, tower_height_2, interval, fresnel = False, curvature = False, landa= 0.0273):

    # Import altitude API
    elevation_data = srtm.get_data()

    # Get initial and final altitudes
    H1 = elevation_data.get_elevation(origin.latitude, origin.longitude)
    H2 = elevation_data.get_elevation(destination.latitude, destination.longitude)
        
    #Initialize the error flag that returns True if there has been any issue with the API
    flag_error = False
    
    #If the API did not find altitudes for either the origin or the destination, then we skip the whole process and return
    #True by default but with the flag_error set to True as well.
    #If we did not get data for any of them, we try with Google's API. If it does not work, we skip with Line of Sight as False
    
    if(pd.isnull(H1)):
        H1 = elevation_google(origin.latitude, origin.longitude, apikey)
           
    if(pd.isnull(H2)):
        H2 = elevation_google(destination.latitude, destination.longitude, apikey)
    
    if(pd.isnull(H1) or pd.isnull(H2)):
        line_of_sight = False
        flag_error  = True
        return (line_of_sight, flag_error, pd.DataFrame({'Lat': [], 'Lon': [], 'Alt': []}), 0, 0)
    
    # Bearing (orientation or azimuth, same thing) needed to go in a straight line from origin to destination
    azimuth = bearing.initial_compass_bearing(origin.latitude, origin.longitude, destination.latitude, destination.longitude)

    # Distance origin-destination
    distance_total = distance.get_distance(origin.latitude, origin.longitude, destination.latitude, destination.longitude)

    # Number of intermediate points where we will evaluate the altitude
    number_segments = int(distance_total / interval)    
    
    
    # Initialize the current point variable and the list of latitudes, longitudes, altitudes and intermediate distances
    current_point = origin
    lats = []
    longs = []
    alts = []
    dis = []
    los = []

    k=4/3
    radio_tierra = 6370000


    # Insert the first set of data
    lats.insert(len(lats), float(origin.latitude))
    longs.insert(len(longs), float(origin.longitude))
    alts.insert(len(alts), float(H1))
    dis.insert(len(dis), 0)
    los.insert(len(los), True)

    # Initialize line_of_sight to True and rest of variables to zero
    line_of_sight = True
    max_alt = 0
    
    delta_height_tower_1 = 0
    delta_height_tower_2 = 0
    max_dt1 = 0
    max_dt2 = 0
    radio_fresnel = 0
    flecha = 0

    # We start evaluating the altitude of all intermediate points between origin and destination
    for y in range(0, number_segments):

        # Evaluate altitude and distance of intermediate point
        destination_intermediate = VincentyDistance(meters=interval).destination(current_point, azimuth)
        altitude = elevation_data.get_elevation(destination_intermediate.latitude, destination_intermediate.longitude)
        di = distance.get_distance(origin.latitude, origin.longitude, destination_intermediate.latitude, destination_intermediate.longitude)

        # If the API returned a valid altitude, we evaluate conditions. Otherwise we skip the process and we will return
        # Line of sight as True and flag_error as True as well
        if altitude:
            recta_los = H1 + tower_height_1 + (H2 - H1 + tower_height_2 - tower_height_1) * di / distance_total
            
            if(distance_total-di >= 0): # Para solucionar error ultima iteración
                if (fresnel is True):
                    radio_fresnel = 0.6 * np.sqrt(landa*di*(distance_total-di)/distance_total)
                else:
                    radio_fresnel = 0
                if (curvature is True):
                    flecha = di*(distance_total-di)/(2*k*radio_tierra)
                else:
                    flecha = 0

            # Geometric conditions to be met if there is NOT line of sight because of this point
            if altitude >= (recta_los-radio_fresnel-flecha):
                #print(recta_los)
                # In this case there is no line of sight and we proceed to calculate geometrically the additional altitude we would
                # need at both ends (separately) in order to have line of sight
                line_of_sight = False
                #delta_height_tower_1 = (di/(distance_total - di))*((altitude - radio_fresnel - flecha - H1)*distance_total/di + H1 - H2 - tower_height_2) - tower_height_1
                #delta_height_tower_2 = (altitude - radio_fresnel - flecha - H1 - tower_height_1)*distance_total/di + H1 - H2 + tower_height_1 - tower_height_2
                delta_height_tower_1 = (di/(distance_total - di))*((altitude + radio_fresnel + flecha - H1)*distance_total/di + H1 - H2 - tower_height_2) - tower_height_1
                delta_height_tower_2 = (altitude + radio_fresnel + flecha - H1 - tower_height_1)*distance_total/di + H1 - H2 + tower_height_1 - tower_height_2
                
                #We update the value of these additional altitudes if it is greater than the additional altitudes calculated previously
                if(delta_height_tower_1 > max_dt1):
                    max_dt1 = delta_height_tower_1
                if(delta_height_tower_2 > max_dt2):
                    max_dt2 = delta_height_tower_2

            # Insert at the end of the lists of points the new values obtained
            lats.insert(len(lats), float(destination_intermediate.latitude))
            longs.insert(len(longs), float(destination_intermediate.longitude))
            alts.insert(len(alts), float(altitude))
            dis.insert(len(dis), float(di))
            los.insert(len(los), bool(line_of_sight))


        # Update intermediate point for next iteration
        current_point = destination_intermediate

    # Once we finish evaluating everything we create a data frame with all the information
    data = [('Lat', lats), ('Long', longs), ('Alt', alts), ('Di', dis), ('LoS', los)]
    df = pd.DataFrame.from_items(data,columns=['Lat','Long', 'Alt', 'Di', 'LoS'])
    
    #df = pandas.DataFrame({'Lat': lats, 'Long': longs, 'Alt': alts, 'Di': dis, 'LoS': los})
    
    #If we do not have enough points (either 0,1 or it failed to get the altitude of 20% of more of the intermediate points)
    if(len(df) <= 1 or len(df)<0.5*number_segments):
        line_of_sight = False
        flag_error = True
    
    # We return line of sight (True or False), flag_error (True or False), data frame with the data, and the max values of the
    # additional height needed.
    return (line_of_sight, flag_error, df, max_dt1, max_dt2)

def elevation_google(latitude, longitude, apikey):
    url = "https://maps.googleapis.com/maps/api/elevation/json"
    request = urllib.request.urlopen(url+"?locations="+str(latitude)+","+str(longitude)+"&key="+apikey)
    response = json.loads(request.read().decode())
    result = response["results"][0]
    elevation = float(result["elevation"])
    return elevation
    


def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
@contextlib.contextmanager
def database(config_path):
    ## Read credentials
    parser= ConfigParser()
    parser.read(config_path)
    database_name   = parser.get('db_config', 'database_name')
    user_name       = parser.get('db_config', 'user_name')
    password        = parser.get('db_config', 'password')
    host            = parser.get('db_config', 'host')
    port            = parser.get('db_config', 'port')
    
    #Open Connection
    conn_string = "postgresql://%s:%s@%s:%s/%s" % (user_name, password, host, port, database_name)
    engine = create_engine(conn_string)
    connection = engine.connect()
    
    yield connection
    
    #Close connection
    connection.close()
    
def transport_by_tower_owner(sql_path, owners, owner_name, mw_radius, fiber_radius):
    # Import data
    query_path = sql_path + '/' + 'tx_by_tower_inputs.sql'
    query_path_all = sql_path + '/' + 'tx_by_tower_all_inputs.sql'

    with open(query_path) as file, open(query_path_all) as file_all, database(config_path) as db:
        query = file.read()
        query_formatted = query.format(schema = schema, table = table_infrastructure, radius = mw_radius, owners = (','.join('\'{0}\''.format(w) for w in owners)), sources_omit = (','.join('\'{0}\''.format(s) for s in sources_omit)))
        df_towers = pd.read_sql_query(query_formatted, con = db)
        
        query_all = file_all.read()
        query_formatted_all = query_all.format(schema = schema, table = table_infrastructure, sources_omit = (','.join('\'{0}\''.format(s) for s in sources_omit)))
        df_towers_all = pd.read_sql_query(query_formatted_all, con = db)
        
    #Calculation of nearest transport point.
    #We calculate for each tower the nearest transport tower (with either radio or fiber) from : Movistar, regional projects and third parties (third parties only if we did not find any previous Movistar transport with line of sight).
    #We prioritize those with line of sight and if there are none, by the smallest additional height to add to any of the two towers: origin or destination.

    #Some simple modifications
    df_towers.distance = df_towers.distance.round()
    df_towers.tower_height_1 = df_towers.tower_height_1.fillna(0)
    df_towers.tower_height_2 = df_towers.tower_height_2.fillna(0)

    #Unique towers to calculate nearest transport (unique IDs in tower_id_1)
    df_towers_unique = df_towers.copy(deep = True)
    df_towers_unique = df_towers_unique['tower_id_1'].drop_duplicates().reset_index().drop('index',1)

    #List of unique IDs
    list_towers_to_process = df_towers_unique['tower_id_1']

    #All existint towers
    list_all_towers = df_towers_all['tower_id']

    #Initialization

    df_towers_all[owner_name + '_transport'] = math.nan
    df_towers_all['distance_' + owner_name + '_transport'] = math.nan
    df_towers_all['line_of_sight_' + owner_name] = False
    df_towers_all['additional_height_tower_1_' + owner_name] = math.nan
    df_towers_all['additional_height_tower_2_' + owner_name] = math.nan
 
    #Those towers that already have radio or fiber are automatically assigned to themselves.
    #The case where, even though the tower does not have fiber and radio, we did not find any tower with transport nearby, will be left as NULL

    #Case where tower has owners' tx
    df_towers_all.ix[
        (~ df_towers_all['tower_id'].isin(list_towers_to_process)) & 
        (df_towers_all['owner'].isin(owners)) &
        ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  | (df_towers_all['tx_3g'] == True)  | 
         (df_towers_all['tx_third_pty'] == True)), owner_name + '_transport'] = df_towers_all.ix[(~ df_towers_all['tower_id'].isin(list_towers_to_process)) &                                                                                                                  
                       (df_towers_all['owner'].isin(owners)) &
                       ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  
                        | (df_towers_all['tx_3g'] == True)  | (df_towers_all['tx_third_pty'] == True)),                                                                                                                    
                       'tower_id'] 
    df_towers_all.ix[
        (~ df_towers_all['tower_id'].isin(list_towers_to_process)) & 
        (df_towers_all['owner'].isin(owners)) & 
        ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  | (df_towers_all['tx_3g'] == True)  |
         (df_towers_all['tx_third_pty'] == True)) , 'distance_' + owner_name + '_transport'] = 0

    df_towers_all.ix[
        (~ df_towers_all['tower_id'].isin(list_towers_to_process)) & 
        (df_towers_all['owner'].isin(owners)) & 
        ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  | (df_towers_all['tx_3g'] == True)  | 
         (df_towers_all['tx_third_pty'] == True)),'line_of_sight_' + owner_name] = True
    df_towers_all.ix[
        (~ df_towers_all['tower_id'].isin(list_towers_to_process)) & 
        (~ df_towers_all['owner'].isin(owners)) & 
        ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  | (df_towers_all['tx_3g'] == True)  | 
         (df_towers_all['tx_third_pty'] == True)),'additional_height_tower_1_' + owner_name] = 0
    df_towers_all.ix[
        (~ df_towers_all['tower_id'].isin(list_towers_to_process)) & 
        (~ df_towers_all['owner'].isin(owners)) & 
        ((df_towers_all['radio'] == True) | (df_towers_all['fiber'] == True)  | (df_towers_all['tx_3g'] == True)  | 
         (df_towers_all['tx_third_pty'] == True)),'additional_height_tower_2_' + owner_name] = 0


    #Number of towers to process:
    n_towers = len(df_towers_unique)

    #Import API for elevation
    elevation_data = srtm.get_data(srtm3 = True)

    #We loop through all the unique tower IDs, for each one of them we will search all possible candidates (all towers within 50 km with transport -separately by Movistar and third parties)
    for x in range(0,n_towers):

        print("Iteration: " + str(x) + "/" + str(n_towers) + "   " + str(datetime.now()))

        #Tower for the current iteration (we search towers nearby this one)
        tower_iteration = df_towers_unique['tower_id_1'][x]
        tower_iteration_info = df_towers.ix[df_towers['tower_id_1'] == tower_iteration]

        #Data frame with all the rows where the tower_id_1 index is that of the tower we are analyzing
        df_iteration = df_towers.copy(deep = True)
        df_iteration = df_iteration.ix[df_iteration['tower_id_1'] == tower_iteration]
        df_iteration = df_iteration.reset_index(drop = True)

        #Number of iterations for this specific tower
        n_iterations = len(df_iteration)

        min_additional_height = 10000 #Big number

        #We loop through each candidate (towers with transport within a 20-km radius)
        for y in range(0, n_iterations):

            #Current tower to evaluate
            tower_2_iteration = df_iteration['tower_id_2'].iloc[y]
            distance_iteration = df_iteration['distance'].iloc[y]

            #If we do not have lat, long for any of the towers (should never happen)
            if(pd.isnull(df_iteration['latitude_1'][y]) or 
               pd.isnull(df_iteration['longitude_1'][y])or 
               pd.isnull(df_iteration['latitude_2'][y]) or 
               pd.isnull(df_iteration['longitude_2'][y])):
                #print("Skip for lack of lat long")
                continue

            #We define origin and destination (towers 1 and 2)
            origin = geopy.Point(df_iteration['latitude_1'][y], df_iteration['longitude_1'][y])
            destination = geopy.Point(df_iteration['latitude_2'][y], df_iteration['longitude_2'][y])

            #We calculate altitudes of both points
            H1 = elevation_data.get_elevation(origin.latitude, origin.longitude, approximate=True)
            H2 = elevation_data.get_elevation(destination.latitude, destination.longitude, approximate=True)

            tower_height_1 = df_iteration['tower_height_1'][y]
            tower_height_2 = df_iteration['tower_height_2'][y]

            #If we did not get data for any of them, we try with Google's API. If it does not work, we skip with Line of Sight as False
            if(pd.isnull(H1)):
                H1 = elevation_google(origin.latitude, origin.longitude, apikey)

            if(pd.isnull(H2)):
                H2 = elevation_google(destination.latitude, destination.longitude, apikey)

            if(pd.isnull(H1) or pd.isnull(H2)):
                line_of_sight = False
                flag_los = True
                continue

            #If they are too close to apply the line of sight algorithm, we assume they do have line of sight since it is a very small distance
            if(df_iteration['distance'][y] < los_interval):
                line_of_sight = True
                flag_los = False
                #print("Skip for distance too small")
                #Case where the candidate tower belongs to Movistar
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, owner_name + '_transport'] = tower_2_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'distance_' + owner_name + '_transport'] = distance_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'line_of_sight_' + owner_name] = line_of_sight
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_1_' + owner_name] = 0
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_2_' + owner_name] = 0
                break


            #If none of the conditions above is true, we evaluate if there is line of sight and save the outputs
            output = check_los(origin, destination, tower_height_1, tower_height_2, los_interval)

            line_of_sight = output[0]
            flag_los = output[1]
            df_points = output[2]
            additional_height_1 = output[3]
            additional_height_2 = output[4]


            #If we did find line of sight, we save the tower. If it is Movistar, we finish the process and stop searching for more candidates. 
            #If we finish searching all Movistars and there is no LoS we go for regional and then for 3rd parties
            if(line_of_sight):
                #print("Entered in line of sight")
                line_of_sight = True
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, owner_name + '_transport'] = tower_2_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'distance_' + owner_name + '_transport'] = distance_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'line_of_sight_' + owner_name] = line_of_sight
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_1_' + owner_name] = additional_height_1
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_2_' + owner_name] = additional_height_2
                break

            #If we did NOT find line of sight but the tower is within a small distance, we save the tower. 
            if(not(line_of_sight) and distance_iteration <= fiber_radius):
                #print("Entered in line of sight")
                line_of_sight = False
                #print("...and then to third-parties chunk")
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, owner_name + '_transport'] = tower_2_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'distance_' + owner_name + '_transport'] = distance_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'line_of_sight_' + owner_name] = line_of_sight
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_1_' + owner_name] = 0
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_2_' + owner_name] = 0
                break


            #If there is not line of sight, we update the best option prioritizing by the smallest additional height.
            if(not(line_of_sight) and (min(additional_height_1, additional_height_2) < min_additional_height)):                
                #print("Entered in NO line of sight") 
                min_additional_height = min(additional_height_1, additional_height_2)
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, owner_name + '_transport'] = tower_2_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'distance_' + owner_name + '_transport'] = distance_iteration
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'line_of_sight_' + owner_name] = line_of_sight
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_1_' + owner_name] = additional_height_1
                df_towers_all.ix[df_towers_all['tower_id'] == tower_iteration, 'additional_height_tower_2_' + owner_name] = additional_height_2
    
    df_towers_all = df_towers_all[['tower_id',
                                    owner_name + '_transport',
                                    'distance_' + owner_name + '_transport',
                                    'line_of_sight_' + owner_name,
                                    'additional_height_tower_1_' + owner_name,
                                    'additional_height_tower_2_' + owner_name]]
    return df_towers_all

def assign_value(x):
    if x.values[0] is None:
        return None
    else:
        return df_output.loc[x.name, x.values[0]]

In [132]:
## Loop through owners dictionary and get best transport towers by owner
query_path = sql_path + '/' + 'tx_by_tower_load_towers.sql'
with open(query_path) as file, database(config_path) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table = table_infrastructure, sources_omit = (','.join('\'{0}\''.format(s) for s in sources_omit)))
    df_output = pd.read_sql_query(query_formatted, db)
        

for i in owners_tx.keys():
    owners = owners_tx[i]['owners']
    owner_name = owners_tx[i]['owner_name']
    df_towers_owner = transport_by_tower_owner(sql_path, owners, owner_name, mw_radius, fiber_radius)
    df_output = df_output.merge(df_towers_owner, on='tower_id', how='left')
    del owners, owner_name, df_towers_owner    
    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:222: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:226: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:231: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:237: DeprecationWarning: 
.ix is deprecated. Please use
.loc fo

Iteration: 0/532   2019-10-22 08:02:40.153829


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:342: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:343: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:344: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:345: DeprecationWarning: 
.ix is deprecated. Please use
.loc fo

Iteration: 1/532   2019-10-22 08:02:40.861293
Iteration: 2/532   2019-10-22 08:02:46.179008
Iteration: 3/532   2019-10-22 08:02:46.380546
Iteration: 4/532   2019-10-22 08:02:48.124737
Iteration: 5/532   2019-10-22 08:02:48.315499
Iteration: 6/532   2019-10-22 08:02:48.560261
Iteration: 7/532   2019-10-22 08:02:48.805153
Iteration: 8/532   2019-10-22 08:02:49.201106
Iteration: 9/532   2019-10-22 08:02:49.863087
Iteration: 10/532   2019-10-22 08:02:50.396722
Iteration: 11/532   2019-10-22 08:02:50.588349
Iteration: 12/532   2019-10-22 08:02:50.783778
Iteration: 13/532   2019-10-22 08:02:50.960787
Iteration: 14/532   2019-10-22 08:02:51.140246
Iteration: 15/532   2019-10-22 08:02:52.220125
Iteration: 16/532   2019-10-22 08:02:52.394662
Iteration: 17/532   2019-10-22 08:02:55.628674
Iteration: 18/532   2019-10-22 08:02:58.831595
Iteration: 19/532   2019-10-22 08:02:58.981729
Iteration: 20/532   2019-10-22 08:03:02.369182


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:354: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:355: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:356: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:357: DeprecationWarning: 
.ix is deprecated. Please use
.loc fo

...

Iteration: 19/381   2019-10-22 08:06:59.665722
Iteration: 20/381   2019-10-22 08:06:59.918191
Iteration: 21/381   2019-10-22 08:06:59.981286
Iteration: 22/381   2019-10-22 08:07:00.028665
Iteration: 23/381   2019-10-22 08:07:00.269269
Iteration: 24/381   2019-10-22 08:07:00.313590
Iteration: 25/381   2019-10-22 08:07:00.360614
Iteration: 26/381   2019-10-22 08:07:00.407937
Iteration: 27/381   2019-10-22 08:07:00.450412
Iteration: 28/381   2019-10-22 08:07:00.710843
Iteration: 29/381   2019-10-22 08:07:00.760203
Iteration: 30/381   2019-10-22 08:07:00.808039
Iteration: 31/381   2019-10-22 08:07:00.858154
Iteration: 32/381   2019-10-22 08:07:00.930981
Iteration: 33/381   2019-10-22 08:07:00.979247
Iteration: 34/381   2019-10-22 08:07:01.412723
Iteration: 35/381   2019-10-22 08:07:01.639962
Iteration: 36/381   2019-10-22 08:07:02.076552
Iteration: 37/381   2019-10-22 08:07:02.515824
Iteration: 38/381   2019-10-22 08:07:02.922376
Iteration: 39/381   2019-10-22 08:07:03.351588
Iteration: 40

Iteration: 192/381   2019-10-22 08:07:43.854013
Iteration: 193/381   2019-10-22 08:07:44.359096
Iteration: 194/381   2019-10-22 08:07:44.402810
Iteration: 195/381   2019-10-22 08:07:44.509141
Iteration: 196/381   2019-10-22 08:07:44.845207
Iteration: 197/381   2019-10-22 08:07:45.020569
Iteration: 198/381   2019-10-22 08:07:45.064477
Iteration: 199/381   2019-10-22 08:07:45.515579
Iteration: 200/381   2019-10-22 08:07:46.456133
Iteration: 201/381   2019-10-22 08:07:46.767285
Iteration: 202/381   2019-10-22 08:07:46.814997
Iteration: 203/381   2019-10-22 08:07:48.519080
Iteration: 204/381   2019-10-22 08:07:49.416692
Iteration: 205/381   2019-10-22 08:07:49.713826
Iteration: 206/381   2019-10-22 08:07:50.029907
Iteration: 207/381   2019-10-22 08:07:50.460341
Iteration: 208/381   2019-10-22 08:07:50.605116
Iteration: 209/381   2019-10-22 08:07:50.653230
Iteration: 210/381   2019-10-22 08:07:50.699438
Iteration: 211/381   2019-10-22 08:07:51.427133
Iteration: 212/381   2019-10-22 08:07:51

Iteration: 363/381   2019-10-22 08:08:39.544705
Iteration: 364/381   2019-10-22 08:08:40.265857
Iteration: 365/381   2019-10-22 08:08:40.998779
Iteration: 366/381   2019-10-22 08:08:41.045652
Iteration: 367/381   2019-10-22 08:08:41.258675
Iteration: 368/381   2019-10-22 08:08:41.571898
Iteration: 369/381   2019-10-22 08:08:41.728427
Iteration: 370/381   2019-10-22 08:08:41.770493
Iteration: 371/381   2019-10-22 08:08:41.989168
Iteration: 372/381   2019-10-22 08:08:42.069968
Iteration: 373/381   2019-10-22 08:08:42.241804
Iteration: 374/381   2019-10-22 08:08:42.503868
Iteration: 375/381   2019-10-22 08:08:42.567355
Iteration: 376/381   2019-10-22 08:08:43.154009
Iteration: 377/381   2019-10-22 08:08:43.539581
Iteration: 378/381   2019-10-22 08:08:43.856868
Iteration: 379/381   2019-10-22 08:08:44.533666
Iteration: 380/381   2019-10-22 08:08:44.852681
Iteration: 0/235   2019-10-22 08:08:45.785764
Iteration: 1/235   2019-10-22 08:08:45.929503
Iteration: 2/235   2019-10-22 08:08:46.08409

Iteration: 157/235   2019-10-22 08:10:49.588905
Iteration: 158/235   2019-10-22 08:10:49.709627
Iteration: 159/235   2019-10-22 08:10:49.817148
Iteration: 160/235   2019-10-22 08:10:50.327985
Iteration: 161/235   2019-10-22 08:10:50.446964
Iteration: 162/235   2019-10-22 08:10:50.885268
Iteration: 163/235   2019-10-22 08:10:51.027530
Iteration: 164/235   2019-10-22 08:10:51.400591
Iteration: 165/235   2019-10-22 08:10:52.013704
Iteration: 166/235   2019-10-22 08:10:52.368949
Iteration: 167/235   2019-10-22 08:10:53.097765
Iteration: 168/235   2019-10-22 08:10:53.487089
Iteration: 169/235   2019-10-22 08:10:53.831364
Iteration: 170/235   2019-10-22 08:10:54.040357
Iteration: 171/235   2019-10-22 08:10:54.971740
Iteration: 172/235   2019-10-22 08:10:55.096773
Iteration: 173/235   2019-10-22 08:10:55.205445
Iteration: 174/235   2019-10-22 08:10:55.359947
Iteration: 175/235   2019-10-22 08:10:56.085018
Iteration: 176/235   2019-10-22 08:10:56.207789
Iteration: 177/235   2019-10-22 08:10:56

Iteration: 47/85   2019-10-22 08:12:09.855626
Iteration: 48/85   2019-10-22 08:12:14.588991
Iteration: 49/85   2019-10-22 08:12:19.968909
Iteration: 50/85   2019-10-22 08:12:20.234621
Iteration: 51/85   2019-10-22 08:12:29.398821
Iteration: 52/85   2019-10-22 08:12:29.470008
Iteration: 53/85   2019-10-22 08:12:30.680444
Iteration: 54/85   2019-10-22 08:12:31.555079
Iteration: 55/85   2019-10-22 08:12:32.267130
Iteration: 56/85   2019-10-22 08:12:32.424669
Iteration: 57/85   2019-10-22 08:12:32.507340
Iteration: 58/85   2019-10-22 08:12:32.614713
Iteration: 59/85   2019-10-22 08:12:32.773500
Iteration: 60/85   2019-10-22 08:12:32.820891
Iteration: 61/85   2019-10-22 08:12:32.978676
Iteration: 62/85   2019-10-22 08:12:33.177297
Iteration: 63/85   2019-10-22 08:12:33.280522
Iteration: 64/85   2019-10-22 08:12:33.442048
Iteration: 65/85   2019-10-22 08:12:33.651883
Iteration: 66/85   2019-10-22 08:12:33.852241
Iteration: 67/85   2019-10-22 08:12:33.898637
Iteration: 68/85   2019-10-22 08:1

In [135]:
## Define third party colnames
third_party_transport_colnames = [(owner_name + '_transport') for owner_name in (third_party_owners)]
third_party_distance_colnames = [('distance_' + owner_name + '_transport') for owner_name in (third_party_owners)]
third_party_line_of_sight_colnames = [('line_of_sight_' + owner_name) for owner_name in (third_party_owners)]
third_party_additional_height_1_colnames = [('additional_height_tower_1_' + owner_name) for owner_name in (third_party_owners)]
third_party_additional_height_2_colnames = [('additional_height_tower_2_' + owner_name) for owner_name in (third_party_owners)]

## Define regional colnames
regional_transport_colnames = [(owner_name + '_transport') for owner_name in (regional_owners)]
regional_distance_colnames = [('distance_' + owner_name + '_transport') for owner_name in (regional_owners)]
regional_line_of_sight_colnames = [('line_of_sight_' + owner_name) for owner_name in (regional_owners)]
regional_additional_height_1_colnames = [('additional_height_tower_1_' + owner_name) for owner_name in (regional_owners)]
regional_additional_height_2_colnames = [('additional_height_tower_2_' + owner_name) for owner_name in (regional_owners)]

In [136]:
df_output_backup = df_output.copy(deep = True)

In [137]:
# Initialize columns
df_output['regional_transport'] = math.nan
df_output['line_of_sight_regional_transport'] = False
df_output['distance_regional_transport'] = math.nan
df_output['additional_height_tower_1_regional'] = math.nan
df_output['additional_height_tower_2_regional'] = math.nan

df_output['third_party_transport'] = math.nan
df_output['line_of_sight_third_party_transport'] = False
df_output['distance_third_party_transport'] = math.nan
df_output['additional_height_tower_1_third_party'] = math.nan
df_output['additional_height_tower_2_third_party'] = math.nan

## Asiign first non-null owner according to input priorization 

if (len(regional_transport_colnames)>0):
    df_output['regional_transport'] = pd.DataFrame(df_output[[s for s in regional_transport_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['line_of_sight_regional_transport'] = pd.DataFrame(df_output[[s for s in regional_line_of_sight_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['distance_regional_transport'] = pd.DataFrame(df_output[[s for s in regional_distance_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['additional_height_tower_1_regional'] = pd.DataFrame(df_output[[s for s in regional_additional_height_1_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['additional_height_tower_2_regional'] = pd.DataFrame(df_output[[s for s in regional_additional_height_2_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)

if (len(third_party_transport_colnames)>0):
    df_output['third_party_transport'] = pd.DataFrame(df_output[[s for s in third_party_transport_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['line_of_sight_third_party_transport'] = pd.DataFrame(df_output[[s for s in third_party_line_of_sight_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['distance_third_party_transport'] = pd.DataFrame(df_output[[s for s in third_party_distance_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['additional_height_tower_1_third_party'] = pd.DataFrame(df_output[[s for s in third_party_additional_height_1_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)
    df_output['additional_height_tower_2_third_party'] = pd.DataFrame(df_output[[s for s in third_party_additional_height_2_colnames]].apply(lambda x: x.first_valid_index(), axis=1)).apply(assign_value,axis=1)

In [144]:
#Export output

query_path = sql_path + '/' + 'transport_by_tower_add_geoms.sql'

with open(query_path) as file, database(config_path) as db:    
    df_output.to_sql(output_table, con=db, if_exists = 'replace', schema = schema, index = False, 
                dtype = {'tower_id': sqlalchemy.types.Integer(),
                         'movistar_transport': sqlalchemy.types.Integer(), 
                         'distance_movistar_transport': sqlalchemy.types.Integer(),
                         'line_of_sight_movistar': sqlalchemy.types.Boolean(), 
                         'additional_height_tower_1_movistar': sqlalchemy.types.Integer(),
                         'additional_height_tower_2_movistar': sqlalchemy.types.Integer(),
                         'third_party_transport': sqlalchemy.types.Integer(),
                         'line_of_sight_third_party_transport': sqlalchemy.types.Boolean(), 
                         'distance_third_party_transport': sqlalchemy.types.Integer(),
                         'additional_height_tower_1_third_party': sqlalchemy.types.Integer(),
                         'additional_height_tower_2_third_party': sqlalchemy.types.Integer(), 
                         (s for s in regional_transport_colnames): sqlalchemy.types.Integer(),
                         (s for s in regional_distance_colnames): sqlalchemy.types.Integer(),
                         (s for s in regional_line_of_sight_colnames): sqlalchemy.types.Boolean(),
                         (s for s in regional_additional_height_1_colnames): sqlalchemy.types.Integer(),
                         (s for s in regional_additional_height_2_colnames): sqlalchemy.types.Integer(), 
                         (s for s in third_party_transport_colnames): sqlalchemy.types.Integer(),
                         (s for s in third_party_distance_colnames): sqlalchemy.types.Integer(),
                         (s for s in third_party_line_of_sight_colnames): sqlalchemy.types.Boolean(),
                         (s for s in third_party_additional_height_1_colnames): sqlalchemy.types.Integer(),
                         (s for s in third_party_additional_height_2_colnames): sqlalchemy.types.Integer()                         
                        })
    query = file.read()
    query_formatted = query.format(schema = schema,
                                   output_table = output_table,
                                   temporary_table = temporary_table,
                                   table_infrastructure = table_infrastructure)
    db.execute(query_formatted)

                       

df_output

KeyError: 'infrastructure_table'